In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [35]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(ChromeDriverManager().install())
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)
    # driver.maximize_window()
    
    
    # =====================================================================================
    # handle cookies
    try:
        cookie = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//button[@aria-label="Accept All"])[1]')))
        if cookie:
            cookie.click()
        else:
            pass
    except Exception as e:
        print("No cookies tab found and the error is {e}")
    
    # =====================================================================================
    # get the login icon and click on it 
    try:
        # get the login button
        login = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '(.//a[@data-wpel-link="internal"])[1]')))
        if login:
            driver.execute_script("arguments[0].scrollIntoView();", login)
            login.click()
        else:
            pass

    except Exception as e:
        print(f"No login tab found and the error is {e}")

# # =====================================================================================
#     # complete logiin info
# # =====================================================================================

    # get the username tab
    try:
        provided_u_name = "fourbrotherstrading@icloud.com"            
        user_name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH,'.//input[@id="username"]')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")

    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/input[@id="password"]')))
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}")
        
    # click login
    try:
        log = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/button[@name="login"]')))
        if log:
            log.click()
            for i in range(2):
                driver.back()
            # time.sleep(5)
    except Exception as e:
        print("No log in")
    # =====================================================================================
    # get total cars
    # =====================================================================================
    try:
        
        num_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@class="product-tile grid-item"]')))
        if num_cars:
            # cars_list = num_cars.split(" ")
            cars =  int(len(num_cars)) # get only number
            print(f"Total {cars} of cars found")
    except Exception as e:
        print(f"No number of cars found and error is {e}")


    # result
    results = []
    car_count = 0
    while car_count < cars: #total_cars
        
        try:
            # car_main_page =WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="products-loop loop grid"]'))) 
            # if car_main_page:
            #     driver.execute_script("arguments[0].scrollIntoView();", car_main_page)
            page_cars= WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@class="product-tile grid-item"]')))
            for i in range(len(page_cars)): #total_cars
                if car_count >= cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/a[@class="product-tile grid-item"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    # time.sleep(1)
                    page_cars[i].click()
                    time.sleep(2)

                    details = {}
                
                    # all details
                    try:
                        car_main_page =WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="products-loop loop grid"]'))) 
                        dets_card = WebDriverWait(car_main_page, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="summary entry-summary"]')))
                        if dets_card:
                            
                            # car name
                            try:
                                title = WebDriverWait(dets_card, 5).until(EC.presence_of_element_located((By.TAG_NAME, 'h1'))).text.strip()
                                if title:
                                    details['title'] = title
                                else:
                                    details['title']="na"
                            except Exception as e:
                                print("No title found")
                                
                            # other_details
                            try:
                                dets = WebDriverWait(dets_card, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="woocommerce-product-details__short-description"]')))
                                if dets:
                                    values = dets.find_elements(By.TAG_NAME, 'p')
                                    if values:
                                        val_lst = [val.text.strip() for val in values]
                                        details['Details'] = val_lst
                                    else:
                                        print("No details found")
                            except Exception as e:
                                print("No details found")
                            
                            # get price
                            try:
                                price = dets = WebDriverWait(dets_card, 5).until(EC.presence_of_element_located((By.XPATH, './/p[@class="price"]'))).text.strip()
                                if price:
                                    details['Price'] = price
                                else:
                                    details['Price'] ="na"
                            except Exception as e:
                                price("No price tag")
                            
                            # images  
                            try:
                                img_card =  WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@class="slick-track"]')))
                                if img_card:
                                    imgs_li = WebDriverWait(img_card, 5).until(EC.presence_of_all_elements_located((By.XPATH, './/li[contains(@class,"slick-slide")]')))
                                    if imgs_li:
                                        imgs_lst = []
                                        for li in imgs_li:
                                            imgs = li.find_element(By.TAG_NAME, 'img').get_attribute("src")
                                            imgs_lst.append(imgs)
                                            
                                        # imgs = [img.get_attribute("src") for img in imgs_li]
                                        details['Images'] = ", ".join(imgs_lst)
                            except Exception as e:
                                print(f"No images {e}")
                            
                            # desc
                            try:                             
                                desc_vals = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, './/div[@id="tab-description"]'))).text.strip()
                                if desc_vals:
                                    details['Desc'] = desc_vals
                                else:
                                    print("No desc")
                                    
                            except Exception as e:
                                print("No desc")                                    
                    except Exception as e:
                        print("no dets card")                            
                    car_count+=1
                    results.append(details)
                    driver.back()
                    time.sleep(2)
                except Exception as e:
                    print("No cars to click")
                    break
            
                

        except Exception as e:
            print(f"No more cars to click {e}")
            break


    df = pd.DataFrame.from_dict(results)
    df.to_csv("house_of_assets.csv", index=False)
    # time.sleep(3)
    driver.quit()

path = "https://www.houseofassets.com/product-category/collectible-cars/"
scrape(path)



Total 18 of cars found
no dets card
no dets card
no dets card
no dets card
No cars to click
No more cars to click Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00CE0283+25139]
	(No symbol) [0x00C6B234]
	(No symbol) [0x00B40300]
	(No symbol) [0x00B7B1B8]
	(No symbol) [0x00BACDF6]
	(No symbol) [0x00BA87E5]
	(No symbol) [0x00BA7D66]
	(No symbol) [0x00B13245]
	(No symbol) [0x00B1379E]
	(No symbol) [0x00B13C1D]
	GetHandleVerifier [0x00FEB143+3215603]
	GetHandleVerifier [0x010022BA+3310186]
	GetHandleVerifier [0x00FFC4D2+3286146]
	GetHandleVerifier [0x00D79C80+654384]
	(No symbol) [0x00C745BD]
	(No symbol) [0x00B12F10]
	(No symbol) [0x00B12AD6]
	GetHandleVerifier [0x01076E5C+3788300]
	BaseThreadInitThunk [0x7552FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x771E82AE+286]
	RtlGetAppContainerNamedObjectPath [0x771E827E+238]



In [36]:
df = pd.read_csv("house_of_assets.csv")
df

EmptyDataError: No columns to parse from file

### image downloader

In [34]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

reg_img = df[["title", "Images"]]

def download_images(data, main_folder="Images"): 
    os.makedirs(main_folder, exist_ok=True)
    
    for index, row in data.iterrows():
        reg_no = row["title"] 
        image_urls = row["Images"]

        # Check if 'Images' column is empty or NaN
        if pd.isna(image_urls) or not isinstance(image_urls, str) or image_urls.strip() == "":
            print(f"Skipping {reg_no} (No image URLs)")
            continue

        image_urls = image_urls.split(", ")  # Split URLs by comma
        
        reg_folder = os.path.join(main_folder, reg_no)
        os.makedirs(reg_folder, exist_ok=True)
        
        for idx, url in enumerate(image_urls):
            url = url.strip()
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url) 
            
            parsed_url = urlparse(url)
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") 
                continue
            
            try:
                response = requests.get(url, stream=True) 
                response.raise_for_status()
                
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"
                file_extension = file_name.split(".")[-1]
                
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"
                
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Call the function
download_images(reg_img)

Downloaded: Images\458 Speciale A\458 Speciale A_1_Screenshot-2024-07-15-at-11.45.21-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_2_Screenshot-2024-07-15-at-11.45.34-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_3_Screenshot-2024-07-15-at-11.45.44-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_4_Screenshot-2024-07-15-at-11.45.53-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_5_Screenshot-2024-07-15-at-11.46.02-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_6_Screenshot-2024-07-15-at-11.44.02-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_7_Screenshot-2024-07-15-at-11.44.18-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_8_Screenshot-2024-07-15-at-11.44.37-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_9_Screenshot-2024-07-15-at-11.44.59-100x100.png
Downloaded: Images\458 Speciale A\458 Speciale A_10_Screenshot-2024-07-15-at-11.45.11-100x100.png
Downloaded: Images\458 Specia